In [16]:
import sys
# path to module files
# sys.path.insert(1, '../python_files/') # insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, './')
# import math as math
from skinning_euclidean import *
from pyassimp import load


In [17]:
# =================================================
# LOADING INITIAL SCENE AND GENERATING TREE DATA
# =================================================

scene = load('./astroBoy_walk.dae')
mesh_id = 3 # astroBoy contains many meshes, we choose the 4th one

mesh = scene.meshes[mesh_id]
v = mesh.vertices
f = mesh.faces
b = mesh.bones
vw = vertex_weight(len(v))
vw.populate(b)


## EUCLIDEAN WAY

In [19]:
# =================================================
# APPLYING EXTRA TRANSFORMATIONS IN JOINTS
# =================================================

transform = False  # True for applying extra transformations in joints

M = initialize_M(b) # initialize M with identity matrices

M[1][0:3,0:3] = eulerAnglesToRotationMatrix([0.3,0.3,0.4]) # apply rotation on the 2nd joint
# M[1][0:3,3] = [5,5,5] # apply translation on the 2nd joint
# M[1] = np.dot(np.diag([2,2,2,1]),M[1])  # apply scaling on the 2nd joint


# =================================================
# EVALUATING NEW VERTICES AFTER SKINNING
# =================================================


MM = read_tree(scene,mesh_id,M,transform)
BB = [b[i].offsetmatrix for i in range(len(b))]
newv = np.zeros([(len(v)),3])

np.save('MM1.npy', np.array(MM, dtype=object), allow_pickle=True)
np.save('BB.npy', np.array(BB, dtype=object), allow_pickle=True)

start = time.time()
for i in range(len(v)):
    for j in range(4):
        if vw.id[i][j] >=0:
            mat = MM[vw.id[i][j]] @ BB[vw.id[i][j]]            
            newv[i] = newv[i] + vw.weight[i][j]*(vertex_apply_M(v[i],mat))
end = time.time()

# =================================================
# PLOTTING
# =================================================


# USING MESH PLOT OUTSIDE JUPYTER
# mp.offline()

print("TIME : ", end-start)
print("TRANSFORMATION = ", transform)
p = mp.plot(newv, f,newv[:, 1],shading={"scale": 2.5,"wireframe":True},return_plot=True)  
# p.save("skinning.html")



TIME :  0.08594703674316406
TRANSFORMATION =  False


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.3864668…

In [20]:
print(len(MM))
print(len(b))

for i in range(len(MM)):
    print (i,"\n" ,MM[i])

21
21
0 
 [[-2.24473494e-01  2.99201650e-02 -9.74020712e-01  2.19862373e-10]
 [ 2.86310336e-01  9.57438688e-01 -3.65724845e-02  8.00826998e+00]
 [ 9.31470809e-01 -2.87081729e-01 -2.23486037e-01 -3.66330986e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
1 
 [[-0.28293441 -0.07966952 -0.95582466  0.03675992]
 [ 0.18683177  0.97287733 -0.13639509  9.18457917]
 [ 0.94076668 -0.21716924 -0.26037563 -0.38934172]
 [ 0.          0.          0.          1.        ]]
2 
 [[-2.82282434e-01 -4.96073501e-02 -9.58047823e-01 -1.44016581e-01]
 [-1.46613426e-01  9.89161345e-01 -8.01971576e-03  1.13921156e+01]
 [ 9.48061745e-01  1.38198891e-01 -2.86495930e-01 -8.82116084e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
3 
 [[-0.20668522  0.1257936  -0.97028714 -0.18131932]
 [-0.17332406  0.97130824  0.16284651 12.13592534]
 [ 0.96293301  0.2018321  -0.17895196 -0.77819605]
 [ 0.          0.          0.          1.        ]]
4 
 [[-2.53697161e-01  9.58218

In [21]:
# =================================================
# APPLYING EXTRA TRANSFORMATIONS IN JOINTS
# =================================================

transform = False  
# transform = True

M = initialize_M(b)

M[1][0:3,0:3] = eulerAnglesToRotationMatrix([0.3,0.3,0.4])
# M[1][0:3,3] = [5,5,5]
# M[1] = np.dot(np.diag([2,2,2,1]),M[1])


# =================================================
# EVALUATING NEW VERTICES AFTER SKINNING
# =================================================


MM2 = read_tree(scene,mesh_id,M,transform)
BB = [b[i].offsetmatrix for i in range(len(b))]
newv = np.zeros([(len(v)),3])

np.save('MM2.npy', np.array(MM2, dtype=object), allow_pickle=True)

start = time.time()
for i in range(len(v)):
    for j in range(4):
        if vw.id[i][j] >=0:
            mat = MM2[vw.id[i][j]] @ BB[vw.id[i][j]]            
            newv[i] = newv[i] + vw.weight[i][j]*(vertex_apply_M(v[i],mat))
end = time.time()

# =================================================
# PLOTTING
# =================================================


# USING MESH PLOT OUTSIDE JUPYTER
# mp.offline()

print("TIME : ", end-start)
print("TRANSFORMATION = ", transform)
p = mp.plot(newv, f,newv[:, 1],shading={"scale": 2.5,"wireframe":True},return_plot=True)  
# p.save("skinning.html")



TIME :  0.07800602912902832
TRANSFORMATION =  False


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.3864668…

In [22]:
# =================================================
# APPLYING EXTRA TRANSFORMATIONS IN JOINTS
# =================================================

MM3 = np.load('MM2.npy', allow_pickle=True)
# BB = np.load('BB.npy', allow_pickle=True)
newv = np.zeros([(len(v)),3])

np.save('MM1.npy', np.array(MM, dtype=object), allow_pickle=True)
np.save('BB.npy', np.array(BB, dtype=object), allow_pickle=True)

start = time.time()
for i in range(len(v)):
    for j in range(4):
        if vw.id[i][j] >=0:
            mat = MM3[vw.id[i][j]] @ BB[vw.id[i][j]]            
            newv[i] = newv[i] + vw.weight[i][j]*(vertex_apply_M(v[i],mat))
end = time.time()

# =================================================
# PLOTTING
# =================================================


# USING MESH PLOT OUTSIDE JUPYTER
# mp.offline()

print("TIME : ", end-start)
print("TRANSFORMATION = ", transform)
p = mp.plot(newv, f,newv[:, 1],shading={"scale": 2.5,"wireframe":True},return_plot=True)  
# p.save("skinning.html")



TIME :  0.12968921661376953
TRANSFORMATION =  False


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.3864668…